## This repo purpose is reconstruct the paper named: "An Unsupervised Learning approach for spectrum allocation in Terahertz communication systems"

## 1. Parameter & System model.

### 1.1. System model: 
- 3D indoor ThzCom system support *nI* users with 1 Access Point
- User distributed uniformly on the floor 
- Vector d: *nI* x 1 vector represent for the distance vector btw user with AP. The element in d are ordered such that d1 < d2 < ... < dnI

#### A. Spectrum of interest:
- Divide the ultra-wide band THz transmission window into 2 areas callded NACSR & PACSR. This paper experiment is on the NACSR
- Focus on multiband based spectrum allocation with ASB 
- Spectrum interested is divided into *nS* sub-bands with unequal bw
- *b* & *f* as the *nSx1* vectors of the BW and the center frequency of the sub-bands.

#### B. Archievable Data Rate
- *r* as the *nSx1* rate vectore of users.
- The rate achieve in the *s*th subband is calculated through the formula (4) in paper.

#### C. Optimal Spectrum Allocation:
- Consider proportionally-fair data rate maximization
- Total data rate = (1xnS)T x log(r)
- Hard to optimize due to the data rate formular rely on parameter b and the molecular absorbption coefficient at f

#### D. DNN architecture:
- 5 layers: 100, 100, 50, 25, 30 neural:
- Activation: ReLU for 4 layers, sigmoid for last layer
- Initial weight: Gaussian random variables with zero mean and unit variance
- Initial biases are set to 0.
- The initial values of λ are set to a small constant of 0.1
- Num interation: 500
- Number of realization of d, nT: 300
- Learning rate: 0.05 for weight, 0.025 for largrange coefficient

## 1. Generate Data

In [8]:
import numpy as np

# gen distance vector d for 15 user
# 1 batch: 300 vector d
# intergration: 500
# => 500*300 vector d
def gen_data():
    data = np.random.uniform((0, 0), (25, 25), (15, 2))
    x_y_ap = [12.5, 12.5]
    # calculate vector d
    _d = np.abs(x_y_ap - data)
    _d = pow(_d,2)
    d = _d[:,0] + _d[:,1]
    
    d = np.sqrt(d + pow(1.7,2))
    return d
X = []
for i in range(300): 
    d = gen_data()
    d.sort()
    X.append(d)
X = np.array(X)
X = X.transpose()

In [9]:
X.shape

(15, 300)

## 2. Deep learning model:


In [ ]:
# calculate data rate & some coefficients

import scipy.integrate as integrate
import math

def molecular_absorption():
    

def lamda():
    ld = ga*gu*(1/No)*pow((3*pow(10,8))/(4*math.pi),2)

def data_rate(p, b, d):
    return math.log((1 + (p*lamda*np.exp(-molecular_absorption*d))/(pow(x, 2)*pow(d,2)*b)),2)
    
upper_bound = f + b / 2
lower_bound = f - b / 2
result = integrate.quad(data_rate, lower_bound, upper_bound)


In [78]:
# Convex Optimization
# NACSR 
# => srn1: 0.557 - 0.671 THz
# => srn2: 0.752 - 0.868 THz
# Approximate frequency:
import scipy.integrate as integrate
import scipy.special as special
import math

def approx_molecular_absorp_1(f):
    #for snr2
    n1 = 10**0.83
    n2 = -10**(-10.04)
    n3 = -10**(-1.23)
    sum_ = n1 + n2*f
    k = (math.e)**sum_ + n3
    return k

def approx_molecular_absorp_2(f):
    #for snr2
    n1 = 10**(0.89)
    n2 = -10**(-10.8)
    n3 = -10**(-1.53)
    sum_ = n1 + n2*f
    k = (math.e)**sum_ + n3
    return k

def bw_to_f(b,bs,fd):
    result = list(b).index(bs)
    fs = fd + sum(b[0:(result-1)]) + bs/2
    return fs

def data_rate(fd, b, bs, ps, ds, Ga, Gu, No, kf):
    fs = bw_to_f(b, bs, fd) # calculate the central frequency of sub-band 
    gamma = Ga * Gu * (1/No) * (((3 * (10**8))/(4*math.pi))**2)
    func = lambda x: math.log((1 + (ps * gamma * ((math.e)**(-kf * ds)))/((x**2)*(ds**2)*bs)),2)
    result = integrate.quad(func , fs - 0.5*bs, fs + 0.5*bs)
    return result

# def total_data_rate(d, p_pred, b_pred):
def loss_function(total_data_rate, p_pred, b_pred, Ptot, btot, lamda1, lamda2):
    loss = tf.reduce_mean(-total_data_rate + lamda1*(p_pred - Ptot) + lamda2*(b_pred - btot))
    return loss


In [79]:
gamma = Ga * Gu * (1/No) * (((3 * (10**8))/(4*math.pi))**2)
gamma

226.8938797705266

In [81]:
import numpy as np


# Multiple layer
d0 = 15 #input
d1 = 100 # 1st layer
d2 = 100 # 2nd layer
d3 = 50 # 3rd layer
d4 = 25 #4th layer
d5 = 30 #output layer

# System params
N = X.shape # batch size
height = 1.7
Ga = 10**3 #30 dbi
Gu = 10**2 #20 dbi
No = 10**(17.4) #dbm/hz
Ptot = 0.0003 #-5dbm
pmax = (5/4)*(Ptot/15)
bmax = 5 #Ghz
fd = 752 #Ghz


# hyper params for Unsuppervised model
learning_rate_weight_bias = 0.05
learning_rate_lagrange = 0.025

# 5 layers
# initial parameters randomly
W1 = 0.01*np.random.normal(size=(d0,d1))
b1 = np.zeros((d1,1))
W2 = 0.01*np.random.normal(size=(d1,d2))
b2 = np.zeros((d2,1))
W3 = 0.01*np.random.normal(size=(d2,d3))
b3 = np.zeros((d3,1))
W4 = 0.01*np.random.normal(size=(d3,d4))
b4 = np.zeros((d4,1))
W5 = 0.01*np.random.normal(size=(d4,d5))
b5 = np.zeros((d5,1))

def relu_derivative(x):
    x[x <= 0] = 0
    x[x > 0] = 1
    return x

def sigmoid_derivative(x):
    return x * (1 - x)

## loop via all data
# batch_size = 300
# interation = 500
def sig(x):
    return 1/(1 + np.exp(-x))
# reshape(X)

for i in range(1):
    ## Feed forward
    Z1 = np.dot(W1.T, X) + b1
    A1 = np.maximum(Z1, 0) #ReLU
    Z2 = np.dot(W2.T, A1) + b2
    A2 = np.maximum(Z2, 0) #ReLU
    Z3 = np.dot(W3.T, A2) + b3
    A3 = np.maximum(Z3, 0) #ReLU
    Z4 = np.dot(W4.T, A3) + b4
    A4 = np.maximum(Z4, 0) #ReLU
    Z5 = np.dot(W5.T, A4) + b5
    Y = sig(Z5)
    print(Y.shape)
    
    ## We need to multiply the result of the backprop with the derivative of Loss function with p and b predicted.
    ## dL/d0 = dY/d0*dL/dY
    
    p_pred = Y[:15,:] * pmax
    print(p_pred.shape)
    b_pred = Y[15:,:] * bmax
    print(b_pred.shape)
    
    eps = 1e-4
    # Derive with p => Calculated through definition of derivative
    p_eps = p_pred + p_pred*eps
    b_eps = b_pred + b_pred*eps
    
    dL_y = np.zeros(Y.shape)
    data_rate_dp_b = np.zeros(Y.shape)
    data_rate_dp_p = np.zeros(Y.shape)
    
    print(b_pred)
    
    for i in range(0,1):
#     for i in range(0,300):
        inp = X[:, i]
        #not chnage
        dpb = np.stack((inp, p_pred[:, i], b_pred[:, i]), axis = 1)
        #p change
        dp_b = np.stack((inp, p_eps[:, i], b_pred[:,i]), axis = 1)
        #b change
        dpb_ = np.stack((inp, p_pred[:, i], b_eps[:, i]),axis = 1)
        ## Can be convert to function => reduce complex
        ## Derivative of data rate with p
        for j in range(0, 15):
            ## Data rate 1
            d = dp_b[j, 0]
            p = dp_b[j, 1]
            b = dp_b[j, 2]
            f = bw_to_f(dp_b[:,2],b,fd)
            kf = approx_molecular_absorp_2(f)
            data_rate_with_d_j = data_rate(fd, dp_b[:,2], b, p, d, Ga, Gu, No, kf)
#             print(d, p, b)
#             print(dp_b[:,2])
#             print(f)
#             print(kf)
#             print(data_rate_with_d_j)
            
            
#             ## Data rate 2
#             d2 = dpb[j, 0]
#             p2 = dpb[j, 1]
#             b2 = dpb[j, 2]
#             f2 = bw_to_f(dpb[:,2],b2,fd)
#             kf2 = approx_molecular_absorp_2(f2)
#             data_rate_with_d_j_2 = data_rate(fd, dpb[:,2], b2, p2, d2, Ga, Gu, No, kf2)
            
#             delta_dr1 = (data_rate_with_d_j - data_rate_with_d_j_2) / (p2*eps)
#             data_rate_dp_p[i,j] = delta_dr1
        
        ## Derivative of data rate with b
#         for j in range(0, 15):
#             ## Data rate 1
#             d = dpb_[j, 0]
#             p = dpb_[j, 1]
#             b = dpb_[j, 2]
#             f = bw_to_f(dpb_[:,2],b,fd)
#             kf = approx_molecular_absorp_2(f)
#             data_rate_with_d_j = data_rate(fd, dpb_[:,2], b, p, d, Ga, Gu, No, kf)
            
#             ## Data rate 2
#             d2 = dpb[j, 0]
#             p2 = dpb[j, 1]
#             b2 = dpb[j, 2]
#             f2 = bw_to_f(dpb[:,2],b2,fd)
#             kf2 = approx_molecular_absorp_2(f2)
#             data_rate_with_d_j_2 = data_rate(fd, dpb[:,2], b2, p2, d2, Ga, Gu, No, kf2)
            
#             delta_dr2 = (data_rate_with_d_j - data_rate_with_d_j_2) / (b2*eps)
#             data_rate_dp_b[i,j] = delta_dr2
        
        
    ### Get loss function for each interations
    
    ### average cost
#     loss = loss_func(Y, lamda1, lamda2)
    
    ## Donot need back propagation
    ## we can calculate via chain rule
    ## train in batch, based on the cost function of this paper => we almost can not initial automation training
    ## so i built a neural network from scratch
    ## Backpropagation
    ## DLoss via weights and bias
    ## dL/d0 = dY/d0*dL/dY
    ## Via chain rule we can calculate
    ## dY/dW5 = dY/dZ5*dZ5/dW5
    
    ##Backprop 1
#     E5 = sigmoid_derivative(Y)
#     dw5 = np.dot(A4, E5.T)
#     db5 = np.sum(E5, axis = 1, keepdims = True)

#     print("Backprop1")
#     print(E5.shape)
#     print(dw5.shape)
#     print(db5.shape)
    
# #     ## Backprop 2
#     E4 = np.dot(W5, E5) * relu_derivative(Z4)
#     dw4 = np.dot(A3, E4.T)
#     db4 = np.sum(E4, axis = 1, keepdims = True)
#     print("Backprop2")
#     print(E4.shape)
#     print(dw4.shape)
#     print(db4.shape)

# #     ##Backprop 3
#     E3 = np.dot(W4, E4) * relu_derivative(Z3)
#     dw3 = np.dot(A2, E3.T)
#     db3 = np.sum(E3, axis = 1, keepdims = True)
#     print("Backprop3")
#     print(E3.shape)
#     print(dw3.shape)
#     print(db3.shape)
    
# #     ##Backprop 4
#     E2 = np.dot(W3, E3) * relu_derivative(Z2)
#     dw2 = np.dot(A1, E2.T)
#     db2 = np.sum(E2, axis = 1, keepdims = True)
#     print("Backprop4")
#     print(E2.shape)
#     print(dw2.shape)
#     print(db2.shape)
    
# #     ##Backprop 5 (input)
#     E1 = np.dot(W2, E2) * relu_derivative(Z1)
#     dw1 = np.dot(X, E1.T)
#     db1 = np.sum(E1, axis = 1, keepdims = True)
#     print("Backprop5")
#     print(E1.shape)
#     print(dw1.shape)
#     print(db1.shape)


(30, 300)
(15, 300)
(15, 300)
[[2.50000596 2.50000641 2.50000574 ... 2.50000606 2.5000059  2.50000574]
 [2.50000173 2.50000184 2.50000164 ... 2.50000164 2.50000181 2.50000157]
 [2.49999127 2.49999122 2.49999083 ... 2.49999153 2.49999065 2.49999149]
 ...
 [2.49999829 2.49999887 2.49999825 ... 2.49999889 2.49999818 2.49999881]
 [2.50000218 2.50000254 2.50000233 ... 2.50000262 2.50000212 2.50000261]
 [2.49999606 2.49999594 2.49999591 ... 2.49999611 2.49999563 2.49999594]]
